# Rabbits and Recurrence Relations

## Problem

A sequence is an ordered collection of objects (usually numbers), which are allowed to repeat. Sequences can be finite or infinite. Two examples are the finite sequence $(\pi,−\sqrt{2},0,\pi)$ and the infinite sequence of odd numbers (1,3,5,7,9,…)(1,3,5,7,9,…). We use the notation anan to represent the nn-th term of a sequence.

A recurrence relation is a way of defining the terms of a sequence with respect to the values of previous terms. In the case of Fibonacci's rabbits from the introduction, any given month will contain the rabbits that were alive the previous month, plus any new offspring. A key observation is that the number of offspring in any month is equal to the number of rabbits that were alive two months prior. As a result, if FnFn represents the number of rabbit pairs alive after the nn-th month, then we obtain the Fibonacci sequence having terms FnFn that are defined by the recurrence relation Fn=Fn−1+Fn−2Fn=Fn−1+Fn−2 (with F1=F2=1F1=F2=1 to initiate the sequence). Although the sequence bears Fibonacci's name, it was known to Indian mathematicians over two millennia ago.

When finding the nn-th term of a sequence defined by a recurrence relation, we can simply use the recurrence relation to generate terms for progressively larger values of nn. This problem introduces us to the computational technique of dynamic programming, which successively builds up solutions by using the answers to smaller cases.

_Given_: Positive integers n≤40n≤40 and k≤5k≤5.

_Return_: The total number of rabbit pairs that will be present after nn months if we begin with 1 pair and in each generation, every pair of reproduction-age rabbits produces a litter of kk rabbit pairs (instead of only 1 pair).

**Sample Dataset**

    5 3
    
**Sample Output**

    19
    
___________________

## Solution

In this notebook we will generalize the classic example of a Fibonacci series to allow for _k_ offspring. This example gives precious insights on recurrence relations and dynamic programming. WWe start by discussing the _k_=1 case, for which the formula is given in the problem statement, and then we will generalize to the case where _k_>1. We will study the running time of the basic recurrent algorithm and improve it using _memoization_. For completeness we will also introduce an iterative approach to calculating Fibonacci numbers. Finally, and as a fun exercice we examine the closed-form solution of $F(n,k)$ using a Fibonacci matrix and eigenvalue decomposition.

The main idea behind a recursive function is to have it call itself, with a smaller input, until it reaches a **base case**. Those are the two crucial characteristics of a recursive function. If the function isn't called on with a smaller input, it will never reach the base case, and it will keep calling itself forever. Similarly if we recursively call the function on smaller input but we don't write a base case, we will be in an infinite loop again.

We start by looking at the regular Fibonacci series where the $n^{th}$ element is given by $F_{n} = F_{n-1}+F_{n-2}$. The function can essentially be written in one line (plus de base cases)! We simply tell our function to return $F_{n-1}+F_{n-2}$. Since we don't know what $F_{n-1}$ and $F_{n-2}$, the function will call itself and set $F_{n-1} = F_{n-2}+F_{n-3}$ and $F_{n-2} = F_{n-3}+F_{n-4}$. The function will keep calling itself until it reaches the base cases, namely $F_{1} = F_{2} = 1$

In [1]:
def fibonacci(n):
    if n < 3:
        return 1
    return fibonacci(n-1) + fibonacci(n-2)

print(fibonacci(6))

8


Let's try to get an idea of the time complexity of our algorithm. As usual we are interested in an upper bound for the runtime of our algorithm as a function of the input. We start by noting that the time needed to compute $F(n)$ is equal to the time needed to compute both $F(n-1)$ and $F(n-2)$ plus the time it takes to perform the operations outside of the recursive calls. These operations are simply a comparison and an addition, which take constant time, so wecan write the time to compute $F(n)$ as: $T(n) = T(n-1)+T(n-2)+O(1)$. Now, we know that computing $F(n-2)$ takes less time than computing $F(n-1)$, but since we are looking for an upper bound we will make the approximation that $T(n-2) \approx T(n-1)$.

So now we can write $T(n) < 2T(n-1) + O(1)$. We now that the base cases take constant time, so we just need to keep calling our function until we reach the base case: $T(n)<2T(n-1)+O(1)<4T(n-2)+O(1)<8T(n-3)+O(1)< \dots < 2^{n-2}T(2)+O(1) \approx O(2^n)$. So our approach takes exponential time! 

Before we move on to ways of improving the runtime of the recursive approach with some dynamic programming, let's generalize for the case where each rabbit can have _k_ offspring. It is easy to see that in order to allow for _k_ offspring we simply need to update our formula to $F(n,k) = F(n-1,k)+k*F(n-2,k)$. Updating our recursive algorithm, we obtained the desired result:


In [2]:
def k_fibonacci(n, k):
    if n < 3:
        return 1
    return k_fibonacci(n-1,k) + k*k_fibonacci(n-2,k)

print(k_fibonacci(5,3))

19


Back to improving the running time of our recursive algorithm. Recall that we have found a basic recursive algorithm to compute the $n^{th}$ Fibonacci number. However our code is not particularly efficient. Consider the case where we want to calculate $F(10)$: our function will call itself to calculate both $F(9)$ and $F(8)$. The first call will then call $F(8)$ and $F(7)$ and so on, until we reach the base cases. Now that we have a value for $F(9)$ we move on to the second call. To calculate $F(8)$ we have to do all the recursions again, even though we calculated $F(8)$ to compute $F(9)$ in the first call! This is highly inefficient since we are calculating the same quantity over and over again.

One very useful and common technique to improve running time is _memoization_. Memoization simply stores the results of different calls in memory so that we don't calculate the same quantity more than once. 

In [3]:
def k_fibonacci_memo(n,k,seen = {1:1, 2:1}):
    if n not in seen:
        seen[n] = k_fibonacci_memo(n-1,k,seen) + k*k_fibonacci_memo(n-2,k,seen)
    return seen[n]

print(k_fibonacci_memo(5,3))

19


We keep track of the values of previous calls in a dictionary, which we initialize with the two base cases. Now at every call we first check to see if what we want is already in the dictionary. If it is we simply read it, otherwise we compute it and save it! The runtime of algorithm is now effectively _O_(n) since looking up a dictionary is _O_(1) and we are computing the _n_ Fibonacci numbers. Let's see how much faster it is, even for a relatively small n:

In [4]:
from time import clock

print("Computing F(40) without memoization...")
start = clock()
k_fibonacci(40,1)
print("Time:", "{:.8f}".format(clock() - start))
print()
print("Computing F(40) with memoization...")
start = clock()
k_fibonacci_memo(40,1,seen = {1:1, 2:1})
print("Time:", "{:.8f}".format(clock() - start))

Computing F(40) without memoization...
Time: 33.20443700

Computing F(40) with memoization...
Time: 0.00010800


Truly impressive. As a final note, we note that both approaches, with and without memoization take _O_(n) space. It is easy to see why in the second case, since we are storing a dictionary with all the previous values. But even without memoization, the program keeps "bookmarks" at each call, waiting for the value to be returned and this also takes up _O_(n) space.

The main idea behind this notebook was to introduce recursive and dynamic programming ideas, using the Fibonacci sequence as an example. However, the fastest and most memory-efficient approach to calculating the $n^{th}$ Fibonacci number is to use a simple loop, in which we only keep track of the two last numbers seen:

In [5]:
def k_fibonacci_iter(n,k):
    a, b = 0, 1
    for i in range(n):
        a, b = b, k*a+b
    return a

print(k_fibonacci_iter(5,3))

19


In [6]:
print("Computing F(40) with memoization...")
start = clock()
k_fibonacci_memo(40,1,seen = {1:1, 2:1})
print("Time:", "{:.8f}".format(clock() - start))
print()
print("Computing F(40) with iteration...")
start = clock()
k_fibonacci_iter(40,1)
print("Time:", "{:.8f}".format(clock() - start))


Computing F(40) with memoization...
Time: 0.00039900

Computing F(40) with iteration...
Time: 0.00010600


_________________
We conclude this notebook with a calculation of a closed-form solution for any $F(n,k)$, using some basic linear algebra techniques.

We starting by writing out the Fibonacci relation in matrix form: $\vec{F_{n+1}}= A(k) \cdot \vec{F_{n}}$, or more explicitely: <p style="text-align: center;"> $\begin{pmatrix} F(n+2,k) \\ F(n+1,k) \end{pmatrix} = \begin{pmatrix} 1 & k \\ 1 & 0 \end{pmatrix} \begin{pmatrix} F(n+1,k) \\ F(n,k) \end{pmatrix} $ </p> 

In order to calculate $F(n,k)$ we need to solve the equation $\vec{F_{n}}= A(k)^{n} \cdot \vec{F_{0}}$. In order to solve this equation we need to express $\vec{F_{0}}$ in terms of $A(k)$'s eigenvectors. We start by calculating the eigenvalues. Solving the characteristic equation $|A-\lambda I| = 0$ quickly gives the eigenvectors 
<p style="text-align: center;"> $\lambda_{1,2} = \dfrac{1 \pm \sqrt{1+4k}}{2}$ </p>

To solve for the eigenvectors $\vec{e_{1,2}}$, we solve $A(k)\cdot \vec{e_{1,2}} = \lambda_{1,2}*\vec{e_{1,2}}$, giving $\vec{e_1} = \begin{pmatrix} \lambda_1 \\ 1 \end{pmatrix}$ and $\vec{e_2} = \begin{pmatrix} \lambda_2 \\ 1 \end{pmatrix}$. We can now decompose $\vec{F_0}$ in terms of matrix's $A(k)$ eigenvectors by solving:
<p style="text-align: center;"> $\vec{F_0} = c_1\vec{e_1}+c_2\vec{e_2} 
= \begin{pmatrix} c_1\lambda_1 + c_2\lambda_2 \\ c_1 + c_2 \end{pmatrix} 
= \begin{pmatrix} 1 \\ 0 \end{pmatrix} $</p>

Giving: <p style="text-align: center;"> $ c_1 = -c_2 = \dfrac{1}{\lambda_1 - \lambda_2} = \dfrac{1}{\sqrt{1+4k}}$ </p>

So: <p style="text-align: center;"> $ \vec{F_0} = \dfrac{1}{\sqrt{1+4k}} (\vec{e_1}-\vec{e_2})$ </p>

Going back to the equation we are looking to solve, $\vec{F_{n}}= A(k)^{n} \cdot \vec{F_{0}}$, we can now take advantage of the decomposition in eigenvectors:

<p style="text-align: center;"> $ \vec{F_n} = \dfrac{1}{\sqrt{1+4k}} A(k)^n \cdot (\vec{e_1}-\vec{e_2})
=  \dfrac{1}{\sqrt{1+4k}}(\lambda_{1}^n\vec{e_1} - \lambda_{2}^n\vec{e_2})$</p>

And, writing down the vectors explicitly:

<p style="text-align: center;"> $ \begin{pmatrix} F(n+1,k) \\ F(n,k) \end{pmatrix}
= \dfrac{1}{\sqrt{1+4k}} \left[ \lambda_{1}^n \begin{pmatrix} \lambda_1 \\ 1 \end{pmatrix} - 
\lambda_{2}^n \begin{pmatrix} \lambda_2 \\ 1 \end{pmatrix}\right]$</p>

Which finally gives:

<p style="text-align: center;"> $ F(n,k) = \dfrac{1}{\sqrt{1+4k}} \left[ \left(\dfrac{1+\sqrt{1+4k}}{2} \right)^{2} - \left(\dfrac{1-\sqrt{1+4k}}{2} \right)^{2}\right]$</p>